<a href="https://colab.research.google.com/github/SaurabhSRP/POC-CNN/blob/main/POC_CNN_model_2_optimizing_it_using_keras_tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Keras tuner documentation link

https://keras.io/keras_tuner/

In [ ]:
pip install -U keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 8.7 MB/s 
     |████████████████████████████████| 1.6 MB 57.3 MB/s 


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

print(tf.__version__)

2.8.2


In [ ]:
fashion_mnist=keras.datasets.fashion_mnist

In [ ]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [ ]:
##lets scale down between 0 to 1
train_images=train_images/255.0
test_images=test_images/255.0

In [ ]:
train_images[0] ##this is how the data looks like so we have to reshape it

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.    

In [ ]:
train_images[0].shape

(28, 28)

In [ ]:
##First we need to reshape the data
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [ ]:
train_images[0].shape

(28, 28, 1)

In [ ]:
##hp - hyperparameter
def build_model(hp):
  model=keras.Sequential([
      keras.layers.Conv2D(
          filters=hp.Int('conv_1_filter',min_value=64,max_value=128,step=16), ##HP.int creates multiple values of between min and max value
          kernel_size=hp.Choice('conv_1_kernel',values=[3,5]), ##Hp.choice - choices between 3,5
          activation='relu',
          input_shape=(28,28,1)
      ),
       keras.layers.Conv2D(
          filters=hp.Int('conv_2_filter',min_value=32,max_value=64,step=16), 
          kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
          activation='relu'
      ),
      keras.layers.Flatten(),
      keras.layers.Dense(
          units=hp.Int('dense_1_units',min_value=32,max_value=128,step=16),
          activation='relu'
      ),
      keras.layers.Dense(10,activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

  return model
       

In [ ]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [ ]:
tuner_search=RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='output',project_name="Mnist Fashion")

In [ ]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 25s]
val_accuracy: 0.8721666932106018

Best val_accuracy So Far: 0.9139999747276306
Total elapsed time: 00h 02m 46s


In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]  ###it has multiple models so we choose the first one [0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 96)        960       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 48)        41520     
                                                                 
 flatten (Flatten)           (None, 27648)             0         
                                                                 
 dense (Dense)               (None, 64)                1769536   
                                                                 
 dense_1 (Dense)             (None, 10)                650       
                                                                 
Total params: 1,812,666
Trainable params: 1,812,666
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1356 - accuracy: 0.9497 - val_loss: 0.2617 - val_accuracy: 0.9145
Epoch 5/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0976 - accuracy: 0.9632 - val_loss: 0.2861 - val_accuracy: 0.9157
Epoch 6/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0692 - accuracy: 0.9743 - val_loss: 0.3419 - val_accuracy: 0.9153
Epoch 7/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0514 - accuracy: 0.9813 - val_loss: 0.3737 - val_accuracy: 0.9138
Epoch 8/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0401 - accuracy: 0.9854 - val_loss: 0.4310 - val_accuracy: 0.9112
Epoch 9/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0302 - accuracy: 0.9896 - val_loss: 0.4574 - val_accuracy: 0.9130
Epoch 10/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0247 - accuracy: 0.9913 - val_loss: 0.5316 - val_accuracy